# Pre-process non-labeled dataset

## Convert to nifti

In [1]:
from pickletools import uint1
import os, glob
import subprocess
import numpy as np

input_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset/'
output_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop
# i=0
for folder1 in sorted(os.listdir(input_dir)):
    # i+=1
    filename = folder1
    for folder2 in os.listdir(input_dir+folder1):
        # if i==10: break
        if 't1' in folder2 and not folder2.startswith('.'):
            input_dicom_folder = input_dir+folder1+'/'+folder2
            
            # output
            output_nifti_folder = output_dir+folder1
            if not os.path.exists(output_nifti_folder):
                os.makedirs(output_nifti_folder)

            # dcm2niix
            # dicom2nifti.dicom_series_to_nifti(input_dicom_folder, output_nifti_folder+filename, reorient_nifti=True)
            cmd = ["dcm2niix", "-f", filename, "-z", "y", "-o", output_nifti_folder, input_dicom_folder]
            process = subprocess.Popen(cmd, stdout=subprocess.PIPE)  # pass the list as input to Popen
            _ = process.communicate()[0]  # the [0] is to return just the output, because otherwise it would be outs, errs = proc.communicate()
            # Dealing with files in that folder
            # for f in glob.glob(input_dir+folder1+'/anat/'+folder1+'_T1.nii.gz'):
            #     os.remove(f)

## Crop eye center

In [ ]:
import numpy
import os,sys
import nibabel as nib
import SimpleITK as sitk
import pandas as pd
import numpy as np

def cropVolumeImage(image,x,y,z,border1,border2,border3):
    
    xSize, ySize, zSize = image.GetSize()
    #xMin,yMin,zMin = image.TransformPhysicalPointToIndex([x-border,y-border,z-border])
    #xMax,yMax,zMax = image.TransformPhysicalPointToIndex([x+border,y+border,z+border])

    #x,y,z = image.TransformPhysicalPointToIndex([x,y,z])
    xMin = x - int(border1/2) 
    xMax = x + int(border1/2) 

    yMin = y - int(border2/2) 
    yMax = y + int(border2/2)  

    zMin = z - int(border3/2) 
    zMax = z + int(border3/2)
    
    # Define limits
    if xMin< 0 :
        xMin = 0  
    if xMax> xSize:
        xMax = xSize

    if yMin< 0 :
        yMin = 0
    if yMax > ySize:
        yMax = ySize

    if zMin< 0 :
        zMin = 0
    if zMax > zSize:
        zMax = zSize

    imageReturn = image[int(xMin):int(xMax),int(yMin):int(yMax),int(zMin):int(zMax)]
    return imageReturn  

# Paths
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_path = main_path + 'non_labeled_dataset_nifti/'
output_path = main_path + 'non_labeled_dataset_nifti_cropped/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Variables to crop 
eye_center = np.array([112, 173, 67]) # Mean coordinates of the center of the right eye (base of the cornea to catch optic nerve too)
croping_cube = np.array([96, 96, 96])

i = 0
for folder1 in sorted(os.listdir(input_path)):
    filename = folder1
    # Path to the folder with images
    input_image_path = input_path + folder1 + '/' + folder1 + '.nii.gz'
    output_image_path = output_path + folder1 + '.nii.gz'
    print(input_image_path)

    input_image = sitk.ReadImage(input_image_path)

    cropped_image = cropVolumeImage(input_image, eye_center[0], eye_center[1], eye_center[2], croping_cube[0], croping_cube[1], croping_cube[2])

    sitk.WriteImage(cropped_image, output_image_path)

    i+=1
    if (i==10):
        break

## Allignment
Allign template's header to subjects spaces

In [13]:
import shutil
import dicom2nifti
import nibabel as nb
from pathlib import Path

# Paths
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_path = main_path + 'non_labeled_dataset_nifti/'
output_path = main_path + 'non_labeled_dataset_nifti_alligned/'
template_path = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0.nii.gz'

template = nb.load(template_path) # load template

i=0
for folder1 in sorted(os.listdir(input_path)):

    # Load t1 image (not alligned)
    t1 = nb.load(input_path + folder1 + '/' + folder1 + '.nii.gz')
    print(t1.affine)

    # Copy affine from template to t1
    nii = nb.Nifti1Image(t1.dataobj, template.affine, t1.header)
    print(t1.affine)

    # Output
    output_image_path = output_path + folder1
    if not os.path.exists(output_image_path):
        os.makedirs(output_image_path)
    nii.to_filename(output_image_path + '/' + folder1 + '_aff.nii.gz')
    
    # Dealing with files in that folder
    # for f in glob.glob(base_dir+folder1+'/input/'+folder1+'_T1_oriented_hdr.nii.gz'):
    #     os.remove(f)

    i+=1
    if (i==10):
        break

[[ 9.97026742e-01  4.70185876e-02  6.10485077e-02 -9.79993744e+01]
 [-4.71064523e-02  9.98889863e-01  0.00000000e+00 -8.24250488e+01]
 [-6.09807707e-02 -2.87578022e-03  9.98134196e-01 -1.76221329e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.97026742e-01  4.70185876e-02  6.10485077e-02 -9.79993744e+01]
 [-4.71064523e-02  9.98889863e-01  0.00000000e+00 -8.24250488e+01]
 [-6.09807707e-02 -2.87578022e-03  9.98134196e-01 -1.76221329e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.99482334e-01  2.09361669e-02  2.44321786e-02 -8.89668427e+01]
 [-2.09424216e-02  9.99780655e-01  0.00000000e+00 -1.01575226e+02]
 [-2.44268253e-02 -5.11668914e-04  9.99701440e-01 -1.71472610e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.99482334e-01  2.09361669e-02  2.44321786e-02 -8.89668427e+01]
 [-2.09424216e-02  9.99780655e-01  0.00000000e+00 -1.01575226e+02]
 [-2.44268253e-02 -5.11668914e-04  9.99701440e-01 -1.714726

## Registration 1 - Full images

In [2]:
import os, glob

# Paths - full image (1st iteration)
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_path = main_path + 'non_labeled_dataset_nifti/'
output_path = main_path + 'non_labeled_dataset_nifti_reg/'
template = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0.nii.gz' # full image template - fixed image (reference)
template_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/sub-29_labels2template5_2.nii.gz'

# i = 0
for folder1 in sorted(os.listdir(input_path)):
    if folder1 not in sorted(os.listdir(output_path)):
    
        # Full images paths
        input_image_path = input_path + folder1 + '/' + folder1 + '.nii.gz' # moving image - full
        output_image_path = output_path + folder1 + '/'

        if not os.path.exists(output_image_path):
            os.makedirs(output_image_path)

        ## antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = 'antsRegistrationSyNQuick.sh -d 3' + \
        ' -m ' + input_image_path                     + \
        ' -f ' + template                             + \
        ' -t ' + 's'                                  + \
        ' -o ' + output_image_path                    + \
        ' -n ' + '16'
        # print(command1)
        # os.system(command1)

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = 'antsApplyTransforms -d 3 '                       + \
        ' -i ' +  template_labels                                    + \
        ' -r ' +  input_image_path                                   + \
        ' -t ' + '[' + output_image_path + '0GenericAffine.mat, 1 ]' + \
        ' -t ' + output_image_path + '1InverseWarp.nii.gz'           + \
        ' -n ' + 'MultiLabel'                                        + \
        ' -o ' +  output_image_path + 'labels.nii.gz'                + \
        ' --float 0 --verbose 1'
        # print(command2)
        # os.system(command2)

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)

        # i+=1
        # if (i==1):
        #     break


--------------------------------------------------------------------------------------
 Mapping parameters
--------------------------------------------------------------------------------------
 ANTSPATH is /opt/ANTs/bin/

 Dimensionality:           3
 Output name prefix:       /mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_reg/2022160101448/
 Fixed images:             /mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0.nii.gz
 Moving images:            /mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti/2022160101448/2022160101448.nii.gz
 Mask images:              
 Initial transforms:       
 Number of threads:        16
 Spline distance:          26
 SyN gradient step:        0.1
 Transform type:           s
 MI histogram bins:        32
 Precision:                d
 Use histogram matching:   0
 Repro                     0
 antsRegistration call:
----------------------------------------------------------

## Crop

In [15]:
import numpy as np
import os,sys
import nibabel as nib
import SimpleITK as sitk
import pandas as pd

# Paths
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_image_path = main_path + 'non_labeled_dataset_nifti/'
input_label_path = main_path + 'non_labeled_dataset_nifti_reg/'
output_path = main_path + 'non_labeled_dataset_nifti_cropped/'

# i=0
for folder1 in sorted(os.listdir(input_label_path)):
    if folder1+'_cropped.nii.gz' not in list(sorted(os.listdir(output_path))):

        image_path = input_image_path + folder1 + '/' + folder1 + '.nii.gz' # image
        labels_path = input_label_path + folder1 + '/labels.nii.gz' # labels
        bound = 15 # boundary for the bounding box (margins)

        image = sitk.ReadImage(image_path)
        all_segments = sitk.ReadImage(labels_path)
        image_x_size, image_y_size, image_z_size = image.GetSize()
        print(f"image_x_size {image_x_size} image_y_size {image_y_size} image_z_size {image_z_size}")

        # Mask
        all_segments_mask = all_segments > 0
        # sitk.WriteImage(all_segments_mask, base_dir+folder+'/input/'+folder+'_labels_mask.nii.gz')

        # Bounding box
        lsif = sitk.LabelStatisticsImageFilter() # It requires intensity and label images
        lsif.Execute(image, all_segments_mask) # Mask! Where all the labels are 1!
        bounding_box = np.array(lsif.GetBoundingBox(1)) # GetBoundingBox(label)
        print(f"Bounding box:  {bounding_box}") # [xmin, xmax, ymin, ymax, zmin, zmax]
        bounding_box_expanded = bounding_box.copy()
        bounding_box_expanded[0::2] -= bound # even indexes
        bounding_box_expanded[1::2] += bound # odd indexes
        print(f"Expanded bounding box: {bounding_box_expanded}")

        # Limits
        if bounding_box_expanded[0] < 0: bounding_box_expanded[0] = 0
        if bounding_box_expanded[1] > image_x_size: bounding_box_expanded[1] = image_x_size
        if bounding_box_expanded[2] < 0: bounding_box_expanded[2] = 0
        if bounding_box_expanded[3] > image_y_size: bounding_box_expanded[3] = image_y_size
        if bounding_box_expanded[4] < 0: bounding_box_expanded[4] = 0
        if bounding_box_expanded[5] > image_z_size: bounding_box_expanded[5] = image_z_size
        print(f"Expanded bounding box after limits: {bounding_box_expanded}")

        # Crop
        image_crop = image[int(bounding_box_expanded[0]):int(bounding_box_expanded[1]), # x
                            int(bounding_box_expanded[2]):int(bounding_box_expanded[3]), # y
                            int(bounding_box_expanded[4]):int(bounding_box_expanded[5])] # z
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        sitk.WriteImage(image_crop, output_path + folder1 + '_cropped.nii.gz')

        # i+=1
        # if (i==1):
        #     break             

image_x_size 176 image_y_size 256 image_z_size 176
Bounding box:  [114 141 157 190  42  71]
Expanded bounding box: [ 99 156 142 205  27  86]
Expanded bounding box after limits: [ 99 156 142 205  27  86]
image_x_size 176 image_y_size 256 image_z_size 176
Bounding box:  [ 98 126 155 190  52  83]
Expanded bounding box: [ 83 141 140 205  37  98]
Expanded bounding box after limits: [ 83 141 140 205  37  98]
image_x_size 176 image_y_size 256 image_z_size 176
Bounding box:  [105 135 160 198  46  81]
Expanded bounding box: [ 90 150 145 213  31  96]
Expanded bounding box after limits: [ 90 150 145 213  31  96]
image_x_size 176 image_y_size 256 image_z_size 176
Bounding box:  [ 96 126 152 190  59  94]
Expanded bounding box: [ 81 141 137 205  44 109]
Expanded bounding box after limits: [ 81 141 137 205  44 109]
image_x_size 176 image_y_size 256 image_z_size 176
Bounding box:  [ 98 127 152 187  46  76]
Expanded bounding box: [ 83 142 137 202  31  91]
Expanded bounding box after limits: [ 83 142 13

## Registration 2 - Cropped images

In [29]:
# Paths - cropped images (2nd iteration)
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_path = main_path + 'non_labeled_dataset_nifti_cropped/'
output_path = main_path + 'non_labeled_dataset_nifti_reg_2/'
template = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0_cropped_15vox.nii.gz' # cropped image template
template_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/Probability_Maps/prob_map_cropped_th0_2.nii.gz'

# i = 0
for folder1 in sorted(os.listdir(input_path)):
    # if folder1.split('_')[0] not in sorted(os.listdir(output_path)):
    if folder1 == '2022160102138_cropped.nii.gz':
        # Cropped images paths
        filename = folder1.split('_')[0]
        input_image_path = input_path + folder1 # moving image - cropped
        output_image_path = output_path + filename + '/'

        if not os.path.exists(output_image_path):
            os.makedirs(output_image_path)

        ## antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = 'antsRegistrationSyNQuick.sh -d 3' + \
        ' -m ' + input_image_path                     + \
        ' -f ' + template                             + \
        ' -t ' + 's'                                  + \
        ' -o ' + output_image_path                    + \
        ' -n ' + '16'
        # print(command1)
        os.system(command1)

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = 'antsApplyTransforms -d 3 '                       + \
        ' -i ' +  template_labels                                    + \
        ' -r ' +  input_image_path                                   + \
        ' -t ' + '[' + output_image_path + '0GenericAffine.mat, 1 ]' + \
        ' -t ' + output_image_path + '1InverseWarp.nii.gz'           + \
        ' -n ' + 'MultiLabel'                                        + \
        ' -o ' +  output_image_path + 'labels.nii.gz'                + \
        ' --float 0 --verbose 1'
        # print(command2)
        os.system(command2)

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)

        # i+=1
        # if (i==99):
        #     break


--------------------------------------------------------------------------------------
 Mapping parameters
--------------------------------------------------------------------------------------
 ANTSPATH is /opt/ANTs/bin/

 Dimensionality:           3
 Output name prefix:       /mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_reg_2/2022160102138/
 Fixed images:             /mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0_cropped_15vox.nii.gz
 Moving images:            /mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_cropped/2022160102138_cropped.nii.gz
 Mask images:              
 Initial transforms:       
 Number of threads:        16
 Spline distance:          26
 SyN gradient step:        0.1
 Transform type:           s
 MI histogram bins:        32
 Precision:                d
 Use histogram matching:   0
 Repro                     0
 antsRegistration call:
----------------------------------------